In [1]:
# importing useful libraries
import os
import scipy.io.wavfile
import scipy.fft
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold, ParameterGrid
from sklearn.ensemble import RandomForestClassifier

In [2]:
# sample importing and feature extraction
N_FOURIER = 3000 # number of features
common_path_dev = "../datasets/free-spoken-digit" + os.path.sep + "dev"
files_dev = os.listdir(common_path_dev)
dataset_dev_features = np.zeros((len(files_dev), N_FOURIER), dtype=float)
dataset_dev_labels = np.zeros((len(files_dev),), dtype=int)
for i in range(len(files_dev)):
    filepath = os.path.join(common_path_dev, files_dev[i])
    dataset_dev_labels[i] = int(files_dev[i].split(".")[0][-1]) # takes class from name
    with open(filepath, "rb") as file_bin:
        tmp = scipy.io.wavfile.read(file_bin)[1]
        dataset_dev_features[i, :] = np.power(np.abs(scipy.fft.fft(tmp, N_FOURIER)), 2)

# creating evaluation set
common_path_eval = "../datasets/free-spoken-digit" + os.path.sep + "eval"
files_eval = os.listdir(common_path_eval)
dataset_eval_features = np.zeros((len(files_eval), N_FOURIER, ), dtype=float)
dataset_eval_numbers = np.zeros((len(files_eval),), dtype=int)
for i in range(len(files_eval)):
    filepath = os.path.join(common_path_eval, files_eval[i])
    dataset_eval_numbers[i] = int(files_eval[i].split(".")[0]) # takes sample number from name
    with open(filepath, "rb") as file_bin:
        tmp = scipy.io.wavfile.read(file_bin)[1]
        dataset_eval_features[i, :] = np.power(np.abs(scipy.fft.fft(tmp, N_FOURIER)), 2)

In [10]:
# split train test & k-fold cross validation
X_train, X_test, y_train, y_test = train_test_split(dataset_dev_features, dataset_dev_labels, test_size=0.2)
kf = KFold(8)
params = {
    "n_estimators": [350],
    "max_depth": [20, 30, 40]
}
best_config = None
max_f1 = 0
# finding best max_depth
for config in ParameterGrid(params):
    score = 0
    n = 0
    for train_indices, validation_indices in kf.split(X_train):
        X_train_curr = X_train[train_indices, :]
        X_valid = X_train[validation_indices, :]
        y_train_curr = y_train[train_indices]
        y_valid = y_train[validation_indices]
        model = RandomForestClassifier(n_estimators=config["n_estimators"], max_depth=config["max_depth"], max_features="sqrt")
        model.fit(X_train_curr, y_train_curr)
        y_pred = model.predict(X_valid)
        score += f1_score(y_valid, y_pred, average="macro")
        n += 1
    score /= n
    if best_config == None or score > max_f1:
        best_config = config
        max_f1 = score

print(best_config)
print(max_f1)

{'max_depth': 20, 'n_estimators': 350}
0.8734598775480431


In [4]:
# model testing
model = RandomForestClassifier(n_estimators=400, max_depth=20, max_features="sqrt")
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
score = f1_score(y_test, y_pred, average="macro")
print("Final score: ", score)

Final score:  0.88393438353178


In [5]:
# final submission
model.fit(dataset_dev_features, dataset_dev_labels) # fitting entire dataset
output = np.zeros((dataset_eval_features.shape[0], 2), dtype=int)
output[:, 0] = dataset_eval_numbers
output[:, 1] = model.predict(dataset_eval_features)
output = output[output[:, 0].argsort()]
with open("predictions.csv", "w") as f:
    f.write("Id,Predicted\n")
    for i in range(output.shape[0]):
        f.write(str(output[i, 0]) + "," + str(output[i, 1]) + "\n")